In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/dsp-project/portrait_emotions_labels.csv')
df

,filename,emotion,label
0,image_06637.jpg,angry,1
1,image_06638.jpg,angry,1
2,image_06639.jpg,disgust,6
3,image_06641.jpg,neutral,7
4,image_06642.jpg,sad,2
...,...,...,...
5831,image_08036.jpg,angry,1
5832,image_08035.jpg,neutral,7
5833,image_08038.jpg,sad,2
5834,image_08028.jpg,fear,4


In [3]:
df_test = pd.read_csv('/kaggle/input/dsp-project/rijks_emotion_cluster1.csv')
df_test

,filename,emotion
0,img_1.jpg,surprised
1,img_10.jpg,disgust
2,img_100.jpg,angry
3,img_101.jpg,fear
4,img_102.jpg,angry
...,...,...
761,img_95.jpg,surprised
762,img_96.jpg,surprised
763,img_97.jpg,happy
764,img_98.jpg,happy


In [4]:
def categorize(row):
    if row['emotion'] == 'angry':
        return 1
    elif row['emotion'] == 'sad':
        return 2
    elif row['emotion'] == 'happy':
        return 3
    elif row['emotion'] == 'fear':
        return 4
    elif row['emotion'] == 'surprised':
        return 5
    elif row['emotion'] == 'disgust':
        return 6
    else:
        return 7

In [5]:
df_test['label'] = df_test.apply(lambda x: categorize(x), axis=1)
df_test

,filename,emotion,label
0,img_1.jpg,surprised,5
1,img_10.jpg,disgust,6
2,img_100.jpg,angry,1
3,img_101.jpg,fear,4
4,img_102.jpg,angry,1
...,...,...,...
761,img_95.jpg,surprised,5
762,img_96.jpg,surprised,5
763,img_97.jpg,happy,3
764,img_98.jpg,happy,3


In [8]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose

In [7]:
training_data = []

for i in range(0, len(df)):
    image, label = df.iloc[i]['filename'], df.iloc[i]['label']
    path = '/kaggle/input/dsp-project/portrait_faces_preprocessed/portrait_faces_preprocessed/' + image
    new_row = [path, int(label)]
    training_data.append(new_row)

In [8]:
print(len(training_data))

5836


In [9]:
train_set = training_data[0:int(len(training_data) * 0.8)]
validation_set = training_data[int(len(training_data) * 0.8):]
print(len(train_set) + len(validation_set))

5836


In [10]:
test_set_filename = []
test_set = []

for i in range(0, len(df_test)):
    image, label = df_test.iloc[i]['filename'].rsplit('/', 1)[-1], df_test.iloc[i]['label']
    path = '/kaggle/input/dsp-project/cropped_images_preprocessed/cropped_images_preprocessed/' + image
    new_row = [path, int(label)]
    test_set_filename.append(image)
    test_set.append(new_row)

In [11]:
print(len(test_set))

766


In [12]:
# Create a map for the labels and their codes
labels = ['angry', 'sad', 'happy', 'fear', 'surprise', 'disgust', 'neutral']
class_map = {}
i = 1
    
for label in labels:
    class_map[i] = label
    i += 1

In [13]:
print(class_map)

{1: 'angry', 2: 'sad', 3: 'happy', 4: 'fear', 5: 'surprise', 6: 'disgust', 7: 'neutral'}


In [14]:
# Class for loading of the data
class own_data(Dataset):
    def __init__(self, train=0, transform=None):
        if train == 0:
            self.data = train_set
        elif train == 1:
            self.data = validation_set
        else:
            self.data = test_set
            
        self.transform = transform
        self.to_pil = ToPILImage()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_id = self.data[idx]
        image = read_image(img_path)
        
        if self.transform:
            image = self.transform(self.to_pil(image))
        
        return image, class_id

In [15]:
from torchvision.models import vgg19

model = vgg19(pretrained=True)
model.load_state_dict(torch.load('/kaggle/input/model-vgg19/vgg19_base'))

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

<All keys matched successfully>

In [16]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False
    
# Change the final layer of VGG-19 Model for Transfer Learning
model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096), 
    nn.ReLU(inplace=True), 
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True), 
    nn.ReLU(inplace=True), 
    nn.Dropout(p=0.5, inplace=False), 
    nn.Linear(in_features=4096, out_features=7, bias=True)
)

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [17]:
preprocess = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(size=224), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [18]:
# Initialize the batch size
batch_size = 64

# Load the train and test data
train_data = own_data(0, preprocess)
validation_data = own_data(1, preprocess)
test_data = own_data(2, preprocess)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [19]:
model.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [10]:
cross = nn.CrossEntropyLoss()

In [ ]:
cross = nn.CrossEntropyLoss()
sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [21]:
# Define the train model function
def train_model(train_loader, model, loss_function, optimizer, batch_size, device=None):
    for batch, (image, label) in enumerate(train_loader):
        # Move labels and images to GPU
        image = image.cuda()
        label = label.cuda()

        optimizer.zero_grad()
        output = model(image)
        label = label - 1
        loss_func = loss_function(output, label)
        loss_func.backward()
        optimizer.step()
        
        if batch % 64 == 0:
            print('[%5d] loss: %.3f' % (batch + 1, loss_func.item()))

In [22]:
# Test function
def validate_model(validation_loader, model, loss_function, batch_size, device=None):
    size = len(validation_loader.dataset)
    validation_loss, correct = 0, 0
    
    with torch.no_grad():
        for image, label in validation_loader:
            # Move labels and images to GPU
            image = image.cuda()
            label = label.cuda()
            
            output = model(image)
            label = label - 1
            validation_loss += loss_function(output, label).item()
            correct += (output.argmax(1) == label).type(torch.float).sum().item()
    
    validation_loss /= batch_size
    correct /= size
    print(f"Test error: \n Accuracy: {(100 * correct):>0.01f}%, Avg loss: {validation_loss:>8f} \n")

In [23]:
device = None

if device == None:
    if torch.cuda.is_available():
        print("CUDA used")
        device = torch.device('cuda')
        model.cuda()

        # Should be > 0
        print(torch.cuda.device_count())

        # Index of device used (can be 0)
        print(torch.cuda.current_device())

        # GPU location
        print(torch.cuda.device(0))

        # Name of GPU
        print(torch.cuda.get_device_name(0))

    else:
        print("CPU used")
        device = torch.device('cpu')

# Force CPU use
# device = torch.device('cpu')

CUDA used
1
0
Tesla P100-PCIE-16GB


In [24]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train()
    train_model(train_loader, model, cross, sgd, batch_size, device=device)
    model.eval()
    validate_model(validation_loader, model, cross, batch_size, device=device)
print("Done!")

Epoch 1
-------------------------------
[    1] loss: 1.976
[   65] loss: 1.330
Test error: 
 Accuracy: 59.5%, Avg loss: 0.351246 

Epoch 2
-------------------------------
[    1] loss: 1.109
[   65] loss: 0.966
Test error: 
 Accuracy: 66.7%, Avg loss: 0.266271 

Epoch 3
-------------------------------
[    1] loss: 1.103
[   65] loss: 0.825
Test error: 
 Accuracy: 68.6%, Avg loss: 0.240385 

Epoch 4
-------------------------------
[    1] loss: 0.772
[   65] loss: 0.597
Test error: 
 Accuracy: 70.4%, Avg loss: 0.227558 

Epoch 5
-------------------------------
[    1] loss: 0.656
[   65] loss: 0.704
Test error: 
 Accuracy: 70.9%, Avg loss: 0.220732 

Epoch 6
-------------------------------
[    1] loss: 0.746
[   65] loss: 0.545
Test error: 
 Accuracy: 71.2%, Avg loss: 0.213660 

Epoch 7
-------------------------------
[    1] loss: 0.424
[   65] loss: 0.512
Test error: 
 Accuracy: 72.3%, Avg loss: 0.214789 

Epoch 8
-------------------------------
[    1] loss: 0.579
[   65] loss: 0.

In [25]:
torch.save(model.state_dict(), '/kaggle/working/model_vgg19')

In [26]:
model.eval()

def test_model(test_loader, model, device=None):
    predictions = []
    
    with torch.no_grad():
        for image, label in test_loader:
            # Move to GPU
            image = image.to(device)
            label = label.to(device)
            
            output = model(image)
            prediction = (output.argmax(1)).cpu().detach().numpy()
            predictions.extend(prediction)
    
    return predictions

In [27]:
predictions = test_model(test_loader, model, device)

In [28]:
predictions = [x + 1 for x in predictions]

In [29]:
rijks_prediction = pd.DataFrame(
    {'filename': test_set_filename,
     'label': predictions
    }
)

rijks_prediction

,filename,label
0,img_1.jpg,3
1,img_10.jpg,4
2,img_100.jpg,2
3,img_101.jpg,3
4,img_102.jpg,6
...,...,...
761,img_95.jpg,4
762,img_96.jpg,3
763,img_97.jpg,3
764,img_98.jpg,3


In [30]:
rijks_prediction.rename(columns = {'label': 'predicted_label'}, inplace=True)
rijks_prediction['actual_label'] = df_test['label']
rijks_prediction

,filename,predicted_label,actual_label
0,img_1.jpg,3,5
1,img_10.jpg,4,6
2,img_100.jpg,2,1
3,img_101.jpg,3,4
4,img_102.jpg,6,1
...,...,...,...
761,img_95.jpg,4,5
762,img_96.jpg,3,5
763,img_97.jpg,3,3
764,img_98.jpg,3,3


In [31]:
correct = (rijks_prediction['predicted_label'] == rijks_prediction['actual_label'])
accuracy = (correct.sum() / correct.size) * 100
accuracy

27.415143603133156

In [32]:
rijks_prediction.to_csv('/kaggle/working/vgg19_predictions_15epochs', index=False)

In [3]:
rijks_prediction = pd.read_csv('predictions/vgg19_predictions_15epochs')
rijks_prediction

,filename,predicted_label,actual_label
0,img_1.jpg,3,5
1,img_10.jpg,4,6
2,img_100.jpg,2,1
3,img_101.jpg,3,4
4,img_102.jpg,6,1
...,...,...,...
761,img_95.jpg,4,5
762,img_96.jpg,3,5
763,img_97.jpg,3,3
764,img_98.jpg,3,3


In [11]:
loss = 0
for i in range(0, len(rijks_prediction)):
    loss += cross(rijks_prediction['predicted_label'].iloc[i], rijks_prediction['actual_label'].iloc[i]).item()

# loss = cross_entropy_loss(np.array(rijks_prediction['actual_label']), np.array(rijks_prediction['predicted_label']))
print("Loss value: ", loss)

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not numpy.int64